In [125]:
import numpy as np
from collections import defaultdict
import re
import json
import matplotlib.pyplot as plt
import warnings
import pandas as pd

In [132]:
friends = pd.read_csv('friends_transcripts.tsv', sep='\t')

In [134]:
pd.set_option('display.max_colwidth', 50) #50 characters in the data frame
data=pd.DataFrame.from_dict(friends)
data.head()

,season_id,episode_id,scene_id,utterance_id,speaker,tokens,transcript
0,s01,e01,c01,u001,Monica Geller,"[['There', ""'s"", 'nothing', 'to', 'tell', '!']...",There's nothing to tell! He's just some guy I ...
1,s01,e01,c01,u002,Joey Tribbiani,"[[""C'mon"", ',', 'you', ""'re"", 'going', 'out', ...","C'mon, you're going out with the guy! There's ..."
2,s01,e01,c01,u003,Chandler Bing,"[['All', 'right', 'Joey', ',', 'be', 'nice', '...","All right Joey, be nice. So does he have a hum..."
3,s01,e01,c01,u004,Phoebe Buffay,"[['Wait', ',', 'does', 'he', 'eat', 'chalk', '...","Wait, does he eat chalk?"
4,s01,e01,c01,u005,unknown,[],NaN


In [81]:
import string
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')

warnings.simplefilter('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/6c06fdb6-5d00-4caf-9b1f-
[nltk_data]     b5d1ef16fd97/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/6c06fdb6-5d00-4caf-9b1f-
[nltk_data]     b5d1ef16fd97/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [136]:
def clean_text(text):

    # lower text
    text = text.lower()

    # tokenize text for space and \n
    text = re.split('\s+|\n',text)
    
    # remove puncutation
    text = [word.strip(string.punctuation) for word in text]

    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]

    # remove stop words
    stop = set(stopwords.words('english'))
    text = [x for x in text if x not in stop]

    # remove empty tokens
    text = [t for t in text if len(t) > 0]

    # pos tag text
    pos_tags = pos_tag(text)

    # create a (word, pos_tag) tuple for each word in text and then lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]

    # remove words with only one letter
    text = [t for t in text if len(t) > 1]

    # join all
    text = " ".join(text)

    return(text)

# using function to clean text data
data['transcript'] = data['transcript'].apply(lambda x: data_cleaning(x))
data.head() 

# Eliminate any row with no entry fordata['Lines'] after cleaning 
data['transcript'].replace('', np.NaN, inplace=True)
data.dropna(inplace=True)

AttributeError: 'float' object has no attribute 'lower'

In [89]:
from sklearn import preprocessing
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [93]:
#implementing count vectorisation
count_vector = CountVectorizer(analyzer=word_tokenize)
X = count_vector.fit_transform(data['transcript'])
print(X.shape)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
#implementing tf-idf word embedding
def tfidf(data):
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2))

    tfidf_result = tfidf.fit_transform(data).toarray()
    tfidf_features = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
    tfidf_features.index = data.index
    return tfidf_features


tfidf_df = tfidf(data_thres['Lines'])
tfidf_df.shape

In [ ]:
#evaluating word embedding techniques
Eval_score_embedding = {'Count Vectorizer': [], 'TF-IDF':[] }
name_embedded_dataframes = {'Count Vectorizer': count_vectorizer_df,'TF-IDF': tfidf_df}

def word_embedding_performance_eval(clf,dataframe):
    Eval_score =  cross_val_score(clf,dataframe, target_thres, cv=5)
    return  Eval_score.mean() 
    
model =  LogisticRegression(max_iter = 1e6)
for dataframe in name_embedded_dataframes.keys():
    Eval_score_embedding [dataframe] = word_embedding_performance_eval(model,name_embedded_dataframes[dataframe])

In [ ]:
from sklearn.preprocessing import LabelEncoder
leb = LabelEncoder()
y = leb.fit_transform(data['speaker'])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
#Training classifier
from sklearn.naive_bayes import MultinomialNB

mnb_clf = MultinomialNB(force_alpha=True)
mnb_clf = mnb.clf.fit(X_train, y_train)

#Testing Classifier

y-pred= mnb_clf.picredt(X_test)
acc = np.sum((y_pred == y_test).astype(int))/y_pred.size
print("accuracy: %4f" % (acc))

In [ ]:
#Finding precision, Recall and F1-score
from sklearn.metrics import precision_recall_fscore_support as prfs

p_micro, r_micro, f1_micro, _ = prfs(y_pred=pred_NB, y_true=y_test, average="micro")
print("Micro Evaluation: Precision: %.4f; Recall: %.4f; F1-Score: %.4f" % (p_micro, r_micro, f1_micro))

p_macro, r_macro, f1_macro, _ = prfs(y_pred=pred_NB, y_true=y_test, average="macro")
print("Macro Evaluation: Precision: %.4f; Recall: %.4f; F1-Score: %.4f" % (p_macro, r_macro, f1_macro))